# Loading the data and process

In [1]:
import pandas as pd

# Load the dataset (Ensure the correct path)
df = pd.read_csv("Raw_labeled_with_gemini.csv")

# Display basic info
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            779 non-null    object
 1   image            506 non-null    object
 2   published        779 non-null    object
 3   description      775 non-null    object
 4   category         779 non-null    object
 5   author           762 non-null    object
 6   image_relation   779 non-null    object
 7   fake_news_label  779 non-null    object
dtypes: object(8)
memory usage: 48.8+ KB
None
                                               title  \
0  Social Security head replaced after clash with...   
1  Richard Linklater and Ethan Hawke Tease New Mo...   
2  Northpoint Commercial Finance Partners with Ya...   
3  Institutions Can Earn Bitcoin Yield With lstBT...   
4  Your 10-Step Guide to Buying a Home, From Star...   

                                               image  \
0  http

In [ ]:

# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Fill missing values
df["image"].fillna("Unknown", inplace=True)
df["author"].fillna("Unknown", inplace=True)
df["description"].fillna("No description available", inplace=True)

# Fix category format (Convert ['category'] → category)
df["category"] = df["category"].str.replace(r"[\[\]']", "", regex=True)

# Convert 'published' column to datetime format
df["published"] = pd.to_datetime(df["published"], errors="coerce")

# Remove duplicates
df.drop_duplicates(inplace=True)

# Save the cleaned dataset
df.to_csv("cleaned_news_data.csv", index=False)

print(" Preprocessing complete! Cleaned data saved as 'cleaned_news_data.csv'")


✅ Preprocessing complete! Cleaned data saved as 'cleaned_news_data.csv'


<ipython-input-2-6be059917ab1>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["image"].fillna("Unknown", inplace=True)
<ipython-input-2-6be059917ab1>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

In [19]:
import os
import re
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel

# Load dataset
file_path = r'C:\Users\rajar\OneDrive\Desktop\New folder\Fake-News\All_Data\Cleaned_news_final1.csv'
df = pd.read_csv(file_path)

# Manual stopwords list
manual_stopwords = set([
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", 
    "it", "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", 
    "these", "they", "this", "to", "was", "will", "with", "we", "you", "your"
])

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = ' '.join([word for word in text.split() if word not in manual_stopwords])
    return text

# Apply text preprocessing
df['cleaned_title'] = df['title'].apply(preprocess_text)
df['cleaned_description'] = df['description'].apply(preprocess_text)

# Tokenize text using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df['tokenized_text'] = df['cleaned_description'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512, return_tensors='pt'))

# Define image transformation pipeline
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define image directory path
IMAGE_DIR = r"C:\Users\rajar\OneDrive\Desktop\New folder\Fake-News\All_Data"  

# Function to load and preprocess images
def load_and_preprocess_image(image_filename):
    if isinstance(image_filename, str):
        image_filename = image_filename.replace("\\", "/")  # Convert Windows-style paths
        image_path = os.path.join(IMAGE_DIR, image_filename)
        if os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
            return image_transform(image)
    return torch.zeros((3, 224, 224))  # Return a blank image tensor if missing

# Apply image preprocessing
df['processed_image'] = df['image_location'].apply(load_and_preprocess_image)

# Convert labels to numerical format
df['label_fake_news'] = df['fake_news_label'].apply(lambda x: 1 if x.lower() == 'fake' else 0)
df['label_image_relation'] = df['image_relation'].apply(lambda x: 1 if x.lower() == 'yes' else 0)

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define PyTorch Dataset class
class NewsDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text_data = self.data.iloc[idx]['tokenized_text']
        image_data = self.data.iloc[idx]['processed_image']
        label_fake_news = torch.tensor(self.data.iloc[idx]['label_fake_news'], dtype=torch.float)
        label_image_relation = torch.tensor(self.data.iloc[idx]['label_image_relation'], dtype=torch.float)
        return text_data, image_data, label_fake_news, label_image_relation

# Create DataLoaders
train_dataset = NewsDataset(train_df)
test_dataset = NewsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define Multimodal Model with Dual Output
class MultiModalFakeNewsModel(nn.Module):
    def __init__(self):
        super(MultiModalFakeNewsModel, self).__init__()
        
        # Text Model (BERT)
        self.text_model = BertModel.from_pretrained("bert-base-uncased")
        self.text_fc = nn.Linear(768, 256)
        
        # Image Model (ResNet-50)
        self.image_model = models.resnet50(pretrained=True)
        self.image_model.fc = nn.Linear(self.image_model.fc.in_features, 256)
        
        # Fusion Layer
        self.fusion_fc = nn.Linear(512, 256)
        
        # Output Layers
        self.fc_fake_news = nn.Linear(256, 1)  # Fake News Classification
        self.fc_image_relation = nn.Linear(256, 1)  # Image-Text Relationship Classification
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, text_input, attention_mask, image_input):
        text_features = self.text_model(text_input, attention_mask=attention_mask).pooler_output
        text_features = self.text_fc(text_features)
        
        image_features = self.image_model(image_input)
        
        combined = torch.cat((text_features, image_features), dim=1)
        fused_features = self.fusion_fc(combined)
        
        output_fake_news = self.fc_fake_news(fused_features)
        output_image_relation = self.fc_image_relation(fused_features)
        
        return self.sigmoid(output_fake_news), self.sigmoid(output_image_relation)

# Initialize Model
model = MultiModalFakeNewsModel()




C:\Users\rajar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
C:\Users\rajar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rajar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also 